In [77]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import os

In [78]:
userAgent ="Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36"
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent
})
    
clist = [1011,1023,1030,1041,1046,1052,1054,1057,1062,1064,1070,1073,1076]

In [79]:
#학과 이름
allCollegelist = []

for c in clist:
    url = "https://www.sookmyung.ac.kr/sookmyungkr/%d/subview.do" % c

    collegeList = soup.select('.college_list')
    
    for college in collegeList:
        r = session.get(url)

        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)

        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text)
        # 학과 이름
        h5El = college.select('h5')
        name = h5El[0].text.strip()
        # 상세보기 URL
        viewEl = college.select('.view')
        view = viewEl[0].attrs['href']
        if view.find('https://www.sookmyung.ac.kr/') == -1:
            view = 'https://www.sookmyung.ac.kr/' + view
        #학과 소개
        infoEl = college.select('.info')
        info = infoEl[0].attrs['href']
        # 학과 홈페이지
        homepageEl = college.select('.homepage')
        homepage = homepageEl[0].attrs['href']
        collegeDict = {"학과 이름":name, "상세보기 URL" : view, "학과 소개" : info, "학과 홈페이지": homepage}

        if os.path.exists("학과pdf") == False:
            os.mkdir("학과pdf")
            
        r = session.get(view)

        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)

        r.encoding = 'utf-8'
        detailSoup = BeautifulSoup(r.text)

        detailInfoEl = detailSoup.select('.college_info_data dl')
        for detailInfo in detailInfoEl:
            dt = detailInfo.select('dt')[0].text.replace(':', "").strip()
            dd = detailInfo.select('dd')[0].text.replace(':', "").strip()
            collegeDict[dt] = dd
        allCollegelist.append(collegeDict)
        
        if info.find('.pdf') >-1:
            r = session.get(info, stream = True)
            if r.status_code == 200:
                r.encoding = 'utf-8'
                with open('%s/%s.pdf' % ('학과pdf',name) , 'wb') as f:
                        f.write(r.raw.read())
                

In [80]:
df = DataFrame(allCollegelist)
df.to_excel('학과목록 리스트.xlsx')
df

,학과 이름,상세보기 URL,학과 소개,학과 홈페이지,전화,팩스,위치,이메일
0,글로벌협력전공,https://www.sookmyung.ac.kr//sookmyungkr/1074/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,http://global.sookmyung.ac.kr/,02-2077-7496,0303-0799-0352,수련교수회관 507,globalservice@sookmyung.ac.kr
1,앙트러프러너십전공,https://www.sookmyung.ac.kr//sookmyungkr/1075/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,http://global.sookmyung.ac.kr/,02-2077-7578,0303-0799-0352,수련교수회관 507,globalservice@sookmyung.ac.kr
2,한국어문학부,https://www.sookmyung.ac.kr//sookmyungkr/1012/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://korean.sookmyung.ac.kr/,02-710-9310,02-2077-7310,순헌관411,km9310@sookmyung.ac.kr
3,역사문화학과,https://www.sookmyung.ac.kr//sookmyungkr/1013/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://history.sookmyung.ac.kr/,"02-710-9361,9367",02-2077-7311,순헌관412,histo-culture@sookmyung.ac.kr
4,프랑스언어ㆍ문화학과,https://www.sookmyung.ac.kr//sookmyungkr/1014/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,http://french.sookmyung.ac.kr/,"02-710-9329,9292",02-2077-7314,순헌관314,french@sookmyung.ac.kr
5,중어중문학부,https://www.sookmyung.ac.kr//sookmyungkr/1015/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://chinese.sookmyung.ac.kr/,02-710-9335,02-2077-7315,순헌관311,china9335@sookmyung.ac.kr
6,독일언어ㆍ문화학과,https://www.sookmyung.ac.kr//sookmyungkr/1016/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://german.sookmyung.ac.kr/,02-710-9342,02-2077-7316,순헌관316B,dokmun@sookmyung.ac.kr
7,일본학과,https://www.sookmyung.ac.kr//sookmyungkr/1017/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://japan.sookmyung.ac.kr/,02-710-9885,02-2077-7313,순헌관602,smjapan@sookmyung.ac.kr
8,문헌정보학과,https://www.sookmyung.ac.kr//sookmyungkr/1018/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://lis.sookmyung.ac.kr/,02-710-9373,02-710-9276,순헌관220,an9377@sookmyung.ac.kr
9,문화관광외식학부문화관광학전공,https://www.sookmyung.ac.kr//sookmyungkr/1020/...,https://admission.sookmyung.ac.kr/enter/pdf/ma...,https://cultureandtourism.sookmyung.ac.kr/,02-710-9969,02-2077-7319,새힘관205,ctour@sookmyung.ac.kr
